Домашняя работа №3

In [77]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from time import sleep
import html
import pandas as pd
import requests
import re

На первом этапе требуется получить все ссылки на книги автора

In [165]:
def get_author_books(driver, author_ids, t_sleep=2):
    url = "https://www.bookvoed.ru/author/books?id={author_id}"
    result = []

    for id in author_ids: 
        driver.get(url.format(author_id=author_ids[id]))
        sleep(t_sleep)
        try:
            click = driver.find_element_by_class_name('wy')
            while  click:
                click.click()
                sleep(t_sleep)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                sleep(t_sleep)
                click = driver.find_element_by_class_name('wy')
        except NoSuchElementException:
            sleep(t_sleep)
        
        sleep(t_sleep)

        books = driver.find_elements_by_class_name("gf")
        for book in books:
            b = book.get_attribute('data-book')
            result.append(int(b))
        sleep(t_sleep)

    return result

In [166]:
# by-url
# books = driver.find_elements_by_class_name("KD")
# for book in books:
#     b = book.find_element_by_css_selector('a').get_attribute('href')
#     result.append(b)

In [167]:
author_id = {
  "Дарья Донцова":  29369,
  "Джеймс Роллинс": 29442,
  "Макс Фрай":      102994,
  "Эрин Хантер":    26149,
  "Дмитрий Емец":   35952
}

In [172]:
with webdriver.Firefox() as driver:
    books = get_author_books(driver, author_id)

In [174]:
print(len(books) == 1774)
# Донцова - 954
# Роллинс - 137
# Фрай - 277
# Хантер - 112
# Емец - 294

True


На втором этапе требуется получить информацию о каждой из книг

In [244]:
def extract_book_info(book_id):
    sleep(0.5)
    url = f"https://www.bookvoed.ru/book?id={book_id}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' +
        'AppleWebKit/537.36 (KHTML, like Gecko) ' +
        'Chrome/42.0.2311.90 Safari/537.36'
    }
    book_html = requests.get(url, headers=headers)
    
#     print(url)
    
    if book_html.status_code != 200:
        raise Exception('Request has failed.')
    
    age = {
        'sv nM': '18+',
        'rv nM': '16+',
        'qv nM': '12+',
        'pv nM': '6+',
        'ov nM': '0+',
    }
    
    soup = BeautifulSoup(book_html.text, 'html.parser')
    book_info = {
        "ID": book_id,
        "Название": soup.find('h1').contents[0].strip(),
        "Цена": soup.find('div', class_='Hu Wu'),
        "Рейтинг": float(soup.find(class_='af')['style'].split()[1].strip('%')),
        "Понравилось": int(soup.find(class_='Ke Me ').contents[0].strip('\n') or 0),
        "В закладки": int(soup.find(class_='Ke Le ff').contents[0].strip('\n') or 0),
        "Не понравилось": int(soup.find(class_='Ke Oe ').contents[0].strip('\n') or 0),
        "Описание": soup.find(class_='lw'),
        "Возраст": age[str(soup.find('div', class_=re.compile(r"...\bnM\b"))).split('"')[1]],
        "Обложка": 'https://www.bookvoed.ru' + soup.find(class_='tf')['src'],
    }
    
    if book_info["Цена"] is not None:
        tmp = book_info["Цена"].text.replace(' ', '').strip('\n').split('pуб.')
        if len(tmp) > 2:
            book_info["Цена"] = float(tmp[1])
        else:
            book_info["Цена"] = float(tmp[0])
    if book_info["Описание"] is not None:
        book_info["Описание"] = book_info["Описание"].contents[0].replace('\n', ' ').strip()

    change_to_int = ['Год', 'Страниц', 'Код', 'Тираж']
    table = soup.find('table', class_='tw')
    rows = table.find_all('tr')
    info = [list(map(lambda x: x.text, row.find_all('td'))) for row in rows]
    for data in info:
        data[0] = data[0].strip(':')
        if data[0] in change_to_int:
            data[1] = int(data[1])
    book_info.update(info)

    return book_info

In [245]:
from multiprocessing import Pool, Lock, Value
from time import sleep

mutex = Lock()
n_processed = Value('i', 0)

def func_wrapper(uid):
    res = extract_book_info(uid) 
    with mutex:
        global n_processed
        n_processed.value += 1
        if n_processed.value % 10 == 0:
            print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
    return res

with Pool(processes=10) as pool:
    result = list(pool.map(func_wrapper, books))

1770 objects are processed...

In [246]:
# mut = [9851982, 6519769, 7694777, 6739699]
# result = list(map(extract_book_info, mut))

df = pd.DataFrame(result)
df.fillna('', inplace=True)
df.sort_values(by=['ID'], inplace=True)
print(df.info())
# df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1774 entries, 953 to 225
Data columns (total 26 columns):
ID                1774 non-null int64
ISBN              1774 non-null object
Автор             1774 non-null object
Авторы            1774 non-null object
В базе            1774 non-null object
В закладки        1774 non-null int64
Возраст           1774 non-null object
Год               1774 non-null object
Издательство      1774 non-null object
Код               1774 non-null int64
Название          1774 non-null object
Не понравилось    1774 non-null int64
Обложка           1774 non-null object
Описание          1774 non-null object
Переводчик        1774 non-null object
Переплёт          1774 non-null object
Понравилось       1774 non-null int64
Производитель     1774 non-null object
Размеры           1774 non-null object
Рейтинг           1774 non-null float64
Серия             1774 non-null object
Страниц           1774 non-null object
Тематика          1774 non-null object

In [247]:
with open('hw_3.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)

In [39]:
# ml = pd.read_csv('hw_3.csv', sep = ',')
# ml